Recently I have been working on integrating Notion with Python. I have been 
doing this as a toy project to work on my programming skills while
also trying to get something useful out of it.

***In this article, I will show you how to automate your todo lists in Notion using Python.***

## Why Notion with Python?

Honestly, there is no simple answer to this. In the main page of the [Notion API](https://developers.notion.com/reference/intro) they show you how to perform requests using the Javascript SDK and cURL. 

What I wanted to do, was to see if I could write something useful with Python just because its my main programming language.

In addition to that, Notion is an extremely versatile and powerful tool for productivity, with which you can do pretty much anything you can think of productivity-wise. 

WIth this simple project I just wanted to have the ability to make updates to my Notion databases from the terminal using a simple command line tool, in order to speed up certain types of workflows. 

## Steps to Automate Todo Lists with Python and the Notion API


Ok, now let's look at the steps to automate todo lists using Python. The goal will be to have the following capabilities:

- Show the current todo list
- Check and uncheck a task in a todo list database in Notion
- Create tasks

To do these we will do the following:
1. Set up a new integration in Notion and generate an API key
2. Create a database for the todo list and get its corresponding id
3. Initialize the variables holding the API keys and relevant database URLs
4. Write a function to read the database
5. Write a function to list all the tasks present in that database
6. Write a function to check and uncheck tasks in the todo list
7. Write a function to add tasks to that database


Ok, now let's get started!

### 1. Set up a new integration in Notion and generate an API key

To setup our integration we just have to headover to:
https://www.notion.so/my-integrations

and click on "New Integration", then copy the generated API token which we will use inside our Python script.

### 2. Create a database for the todo list and get its corresponding id

To create a database we just open a page in our notion workspace and type in:

```/database```

which should trigger the functionality of database inline or database as a full page. Select the database as a full page and copy the following from that page's corresponding URL:

```https://www.notion.so/<workspace_name>/<ID_TO_COPY>?v=<ANOTHER_BIG_NUMBER>```

We should copy the id number that comes after the workspace name and before the 
`>?` symbols.


### 3. Initialize the variables holding the API keys and relevant database URLs

TO do this, we must first export the corresponding keys to our environment as to 
avoid having them publicly present inside our code, in Linux you can just edit your
`~/.bashrc` folder and write 2 lines as the following:

```
export NOTION_TOKEN="notion integration api key"
export NOTION_TASK_DATABASE_ID="database id"
```

Replace these values with the ones relevant to your project.

Now, we can access these values from inside our Python script or jupyter notebook:

In [1]:
import requests, json
import os
# Custom imports
from src.utils import *

token = os.environ["NOTION_TOKEN"]
databaseId = os.environ["NOTION_TASK_DATABASE_ID"]
update_DB_URL = f'https://api.notion.com/v1/databases/{databaseId}'

headers = {
    "Authorization": "Bearer " + token,
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28"}

Here we are setting up the following:

- `token` - NOTION api key we need to use the integration, 
- `databaseID` - The database ID to access our todo list database
- `update_DB_URL` - The corresponding url to that database so we can send requests to update it or retrieve data from it.
- `headers` - A dictionary to set up the authentication for sending requests to the Notion API.

### 4. Write a function to read the database

In [2]:
def readDatabase(databaseId, headers):
    readUrl = f"https://api.notion.com/v1/databases/{databaseId}/query"

    res = requests.request("POST", readUrl, headers=headers)
    data = res.json()
    print(res.status_code)

    with open('./db.json', 'w', encoding='utf8') as f:
        json.dump(data, f, ensure_ascii=False)
    
    return data


Here we define a function called `readDatabase` which takes two parameters: `databaseId` and `headers`.

We start by defining the URL to query the database:

```readUrl = f"https://api.notion.com/v1/databases/{databaseId}/query"```

which creates a URL to query the Notion API for a specific database ID passed as `databaseId`.

Then, we send an HTTP POST request to the URL created in the previous line, with the headers passed as headers. The response is stored in the res variable. After that we extract the JSON data from the response and stores it in the data variable.


```
res = requests.request("POST", readUrl, headers=headers)
data = res.json()
print(res.status_code)
```

We follow to create or open a `"db.json"` in the current directory in write mode. The with statement ensures that the file is properly closed when the block is exited. We write the JSON data (stored in the data variable) to the file opened. And finally, we return the extracted JSON data to the caller of the `readDatabase` function.

```
with open('./db.json', 'w', encoding='utf8') as f:
    json.dump(data, f, ensure_ascii=False)

return data
```


### 5. Write a function to list all the tasks present in that database

In [3]:
def get_tasks_from_db(db_data):
    """
    Gets a task from my task database.
    """
    length_of_table = len(db_data["results"])
    content_list = []
    for i in range(length_of_table):
        if len(db_data["results"][i]["properties"]["Task"]["title"])!=0:
            task_name = db_data["results"][i]["properties"]["Task"]["title"][0]["plain_text"]
            print(f"{i} - {task_name}")

The `get_tasks_from_db` function takes a database `db_data` as input and performs the following steps:

- Calculates the number of tasks in the database.
- Creates an empty list to store task names.
- Loops through each task in the database, checking if it has a title.
- Extracts the name of each task and prints its index and name to the console.

### 6. Write a function to check and uncheck tasks in the todo list

In [4]:
def check_task(task_page_data, task_page_id, headers, check_status=True):
    updateURL = f'https://api.notion.com/v1/pages/{task_page_id}'
    task_page_data["properties"]["Check"]["checkbox"]=check_status
    task_name = task_page_data["properties"]["Task"]["title"][0]["plain_text"]
    updated_task_page = json.dumps(task_page_data)
    res = requests.request("PATCH", updateURL, headers=headers, data=updated_task_page)
    #print(res.text)
    print(f"Task: {task_name} ✅")

We create a function called `check_task()` which updates the status of a specific task page in Notion's database. It takes four parameters: 
- `task_page_data`
- `task_page_id`
- `headers`
- `check_status`. 
 
The function updates the "Check" property of the task page data with the value of check_status. 
The updated task page data is converted to a JSON string and sent to the Notion API using an HTTP PATCH request. Finally, a message is printed to the console indicating that the task has been checked.

## 7. Write a function to add tasks to that database

In [5]:
def create_task(task_title, tasks_databaseId, headers):
    update_DB_URL = f'https://api.notion.com/v1/databases/{tasks_databaseId}'
    update_PAGE_URL = "https://api.notion.com/v1/pages"
    # Define the properties of the new page
    new_page = {
        "Task": {
            "title": [
                {
                    "text": {
                        "content": f"{task_title}"
                    }
                }
            ]
        }
    }

    # Define the parent of the new page
    parent = {
        "database_id": tasks_databaseId
    }

    # Combine the new page properties and parent into a request body
    data = {
        "parent": parent,
        "properties": new_page
    }

    # Send the request to create the new page
    response = requests.post(update_PAGE_URL, headers=headers, data=json.dumps(data))

    # Print the response content and status code
    print(response.status_code)

The function `create_task()` takes three parameters: 
- `task_title` - the title of the task being created
- `tasks_databaseId` - the id of the tasks database
- `headers`.  - the header for the authentication of the request

It creates a URL to update the specified Notion database, sets the URL for creating a new page in Notion, creates a dictionary that represents the new page to be created in Notion, creates a dictionary that specifies the parent of the new page, combines the new page dictionary and parent dictionary into a single request body that will be sent to the Notion API to create the new page, then sends an HTTP POST request to the Notion API to create a new page with the specified title in the specified database, and prints the status code of the response received from the Notion API after creating the new page.

### Wrap Everything into a CLI tool

Now that we have all the necessary ingredients, let's wrap this code into a neat cli tool to use it from the terminal:

In [ ]:
import requests, json
import os
# Custom imports
from src.utils import *
import argparse

token = os.environ["NOTION_TOKEN"]
tasks_databaseId = os.environ["NOTION_TASK_DATABASE_ID"]
update_DB_URL = f'https://api.notion.com/v1/databases/{tasks_databaseId}'

headers = {
    "Authorization": "Bearer " + token,
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28"}

if __name__=="__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--get", type=str, default="")
    parser.add_argument("--check_task", type=int, default=None)
    parser.add_argument("--uncheck_task", type=int, default=None)
    parser.add_argument("--add_task", type=str, default=None, 
                        help="Create a task in your task db.")
    args = parser.parse_args()
    
    if args.get=="todos":
        db_data = readDatabase(tasks_databaseId, headers)
        print(get_tasks_from_db(db_data))
    
    if args.check_task!=None:
        task_num=args.check_task
        task_status = True
        page_data = get_page_in_database(tasks_databaseId, headers, task_num)
        page_id = page_data["id"]
        check_task(page_data, page_id, headers, check_status=task_status)
    
    if args.uncheck_task!=None:
        task_num=args.uncheck_task
        task_status = False
        page_data = get_page_in_database(tasks_databaseId, headers, task_num)
        page_id = page_data["id"]
        check_task(page_data, page_id, headers, check_status=task_status)
    
    if args.add_task!=None:
        task_title = args.add_task
        create_task(task_title, tasks_databaseId, headers)
        


Here, we simply use Python's `argparse` module to wrap all the functions written into a cli tool that we can access from the terminal.

---

## Testing the Automation

To finally test this automation, all we have to do is call the cli tool from the terminal giving the relevant arguments:

```
python ./notion_py_cli.py --get todos

# Output

0 - Workflow test task 1
1 - Edit the Youtube short / Tiktok
2 - Gym weights
3 - Learning: math course
4 - Notion Todo List Sync
.
.
.
```



![](2023-02-12-17-51-58.png)

Now, for checking and unchecking a task:

Check:

```
python ./notion_py_cli.py --check_task 0

Output:

200
Task: Workflow test task 1 ✅
```

![](2023-02-12-17-53-49.png)

Uncheck:

```
python ./notion_py_cli.py --uncheck_task 0

# Output

200
Task: Workflow test task 1 ✅
```
![](2023-02-12-17-55-05.png)


There you have it! We managed to automate simple Notion todo lists with Python!

---

## Extending the Automation

There is a lot that we could do starting from here, we could sync this automation to our github workflows, or even have our local files sync up with Notion through Python, in order to generate automatic reports from code and jupyter notebooks for example. 

ON upcoming article I will try to address some more interesting automations we could use!


If you liked this post, join [Medium](https://lucas-soares.medium.com/membership), subscribe to my [Youtube channel](https://www.youtube.com/channel/UCu8WF59Scx9f3H1N_FgZUwQ) and my [newsletter](https://lucas-soares.medium.com/subscribe). Thanks and see you next time! :)

